In [1]:
import numpy as np
import pandas as pd
import graphviz
import lingam
from lingam.utils import make_dot, print_causal_directions, print_dagc

print([np.__version__, pd.__version__, graphviz.__version__, lingam.__version__])

np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)




['1.26.4', '2.2.2', '0.20.3', '2.0.6']


In [33]:

X = pd.read_csv('./examples/data/data_benchmark_1/fMRI_processed_by_Nauta/returns/timeseries6.csv')
print(X)



import time
start_time = time.time()
model = lingam.VARLiNGAM(lags=2)
model.fit(X)
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")


print(model.causal_order_)


# B0
B0_result=model.adjacency_matrices_[0]
print(B0_result)


# B1
B1_result=model.adjacency_matrices_[1]
print(B1_result)

# # B2
# B2_result=model.adjacency_matrices_[2]
# print(B2_result)



def precision_recall_f1(matrix1, matrix2):
    assert matrix1.shape == matrix2.shape, "Matrices must have the same shape"

    # Convert matrices to boolean: True for non-zero, False for zero
    bool_matrix1 = matrix1 != 0
    bool_matrix2 = matrix2 != 0

    # Calculate True Positives (TP), False Positives (FP), False Negatives (FN), True Negatives (TN)
    TP = np.sum(np.logical_and(bool_matrix1, bool_matrix2))
    print(f"TP:{TP}")
    FP = np.sum(np.logical_and(np.logical_not(bool_matrix1), bool_matrix2))
    print(f"FP:{FP}")
    FN = np.sum(np.logical_and(bool_matrix1, np.logical_not(bool_matrix2)))
    print(f"FN:{FN}")
    TN = np.sum(np.logical_and(np.logical_not(bool_matrix1), np.logical_not(bool_matrix2)))
    print(f"TN:{TN}")

    # Compute precision, recall, and f1-score
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP+TN)/(TP+TN+FP+FN)

    return precision, recall, f1_score,accuracy




# Load the CSV file
#file_path = 'examples/data/data_benchmark_1/FinanceCPT/ground_truth/random-rels_20_1C.csv'  # Change this to your actual file path
file_path = 'examples/data/data_benchmark_1/fMRI_processed_by_Nauta/ground_truth/sim6_gt_processed.csv'  # Change this to your actual file path
data = pd.read_csv(file_path, header=None)
data.columns = ['cause', 'effect', 'edge']

# Determine the size of the causality matrix
num_causes = data['cause'].max() + 1
num_effects = data['effect'].max() + 1
dim=max(num_causes, num_effects)
# Create an empty matrix of appropriate size
causality_matrix = np.zeros((dim, dim), dtype=int)

# Populate the matrix with the 'edge' values
for _, row in data.iterrows():
    causality_matrix[row['effect'], row['cause']] = row['edge']

# Display the causality matrix
print(causality_matrix)
print(causality_matrix.shape)




result = np.zeros_like(model.adjacency_matrices_[0])
for idx, adj_matrix in enumerate(model.adjacency_matrices_):
    #result=np.maximum(result, adj_matrix)
    result += np.abs(adj_matrix)
    print(f"B{idx} result:")
    print(adj_matrix)
#result = np.abs(B0_result) + np.abs(B1_result)# + np.abs(B2_result)
print("B result:")
print(result)





# Assume a threshold to binarize the result
threshold = 0.1
result = (result >= threshold).astype(int)
print("B result:")
print(result)


precision_B0, recall_B0, f1_score_B0, accuracy_B0 = precision_recall_f1(causality_matrix, result)
print(f"Precision B0: {precision_B0}")
print(f"Recall B0: {recall_B0}")
print(f"F1-score B0: {f1_score_B0}")
print(f"Accuracy B0: {accuracy_B0}")


# Compute the structural hamming distance
def structural_hamming_distance(matrix1, matrix2):
    assert matrix1.shape == matrix2.shape, "Matrices must have the same shape"

    # Convert matrices to boolean: True for non-zero, False for zero
    bool_matrix1 = matrix1 != 0
    bool_matrix2 = matrix2 != 0

    # Compute the Hamming distance
    distance = np.sum(bool_matrix1 != bool_matrix2)

    return distance



shd_B0 = structural_hamming_distance(causality_matrix, result)
#print(causality_matrix)
#print(B0_result)
print(f"Structual Hamming Distance of B0: {shd_B0}")
error_rate_B0=shd_B0/B0_result.shape[0]**2
print(f"Error rate B0: {error_rate_B0}")
print(f"Accuracy B0: {1-error_rate_B0}")











result = np.zeros_like(model.adjacency_matrices_[0])
for idx, adj_matrix in enumerate(model.adjacency_matrices_):
    result=np.maximum(result, adj_matrix)
    #result += np.abs(adj_matrix)
    print(f"B{idx} result:")
    print(adj_matrix)
#result = np.abs(B0_result) + np.abs(B1_result)# + np.abs(B2_result)
print("B result:")
print(result)





# Assume a threshold to binarize the result
threshold = 0.1
result = (result >= threshold).astype(int)
print("B result:")
print(result)


precision_B0, recall_B0, f1_score_B0, accuracy_B0 = precision_recall_f1(causality_matrix, result)
print(f"Precision B0: {precision_B0}")
print(f"Recall B0: {recall_B0}")
print(f"F1-score B0: {f1_score_B0}")
print(f"Accuracy B0: {accuracy_B0}")


# Compute the structural hamming distance
def structural_hamming_distance(matrix1, matrix2):
    assert matrix1.shape == matrix2.shape, "Matrices must have the same shape"

    # Convert matrices to boolean: True for non-zero, False for zero
    bool_matrix1 = matrix1 != 0
    bool_matrix2 = matrix2 != 0

    # Compute the Hamming distance
    distance = np.sum(bool_matrix1 != bool_matrix2)

    return distance



shd_B0 = structural_hamming_distance(causality_matrix, result)
#print(causality_matrix)
#print(B0_result)
print(f"Structual Hamming Distance of B0: {shd_B0}")
error_rate_B0=shd_B0/B0_result.shape[0]**2
print(f"Error rate B0: {error_rate_B0}")
print(f"Accuracy B0: {1-error_rate_B0}")



            0        1         2        3         4         5         6  \
0    -2.69100 -1.95910 -2.552400 -1.25590 -1.339600  2.101700 -1.059000   
1    -1.69740  3.19930  0.043200 -0.94342 -4.372200 -0.646490 -0.855830   
2     0.90524  4.19570  1.362800 -0.89663 -1.273800 -1.525000  0.201560   
3     3.12950  3.01960  0.008149 -1.12460 -0.125310 -1.298800 -0.684750   
4     2.22200 -2.78580 -0.721280 -0.59846  4.871900 -0.999250 -0.282030   
...       ...      ...       ...      ...       ...       ...       ...   
1195 -0.43135 -4.33950 -2.437800  2.70770  3.537600 -0.174670  1.650500   
1196 -6.28480 -0.79428 -3.976800 -0.47824 -3.462500 -0.670610  0.496270   
1197 -3.68720 -0.48170  0.110430 -1.67120 -4.018600  0.076852 -0.032055   
1198 -0.84579 -2.76220 -1.165800 -1.14920 -0.721700 -1.647100 -0.113880   
1199  2.19910 -4.00140  2.514400 -2.04100  0.031647 -1.483500  1.455500   

            7        8       9  
0    -0.98590 -2.14800 -1.7483  
1    -1.97830 -0.84755  1.5025  


In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import lingam
import time

# 定义处理单个数据集的函数
def process_dataset(timeseries_file, groundtruth_file, threshold=0.1):
    
    X = pd.read_csv(timeseries_file)
    n_samples = X.shape[0]
    model = lingam.VARLiNGAM(lags=2)

    start_time = time.time()
    model.fit(X)
    end_time = time.time()
    execution_time = end_time - start_time

    adjacency_matrices = model.adjacency_matrices_

    result = np.zeros_like(adjacency_matrices[0])
    for idx, adj_matrix in enumerate(adjacency_matrices):
        result += np.abs(adj_matrix)

    result = (result > threshold).astype(int)

    data = pd.read_csv(groundtruth_file, header=None)
    data.columns = ['cause', 'effect', 'edge']

    num_causes = data['cause'].max() + 1
    num_effects = data['effect'].max() + 1
    dim = max(num_causes, num_effects)

    causality_matrix = np.zeros((dim, dim), dtype=int)
    for _, row in data.iterrows():
        causality_matrix[row['effect'], row['cause']] = row['edge']

    precision, recall, f1_score, accuracy = precision_recall_f1(causality_matrix, result)
    shd = structural_hamming_distance(causality_matrix, result)
    error_rate = shd / (dim ** 2)

    return {
        "features": dim,
        "n_samples": n_samples,
        "SHD": shd,
        "error rate": error_rate,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1score": f1_score,
        "total excution time": execution_time
    }

# 定义计算指标的辅助函数
def precision_recall_f1(matrix1, matrix2):
    assert matrix1.shape == matrix2.shape, "Matrices must have the same shape"

    bool_matrix1 = matrix1 != 0
    bool_matrix2 = matrix2 != 0

    TP = np.sum(np.logical_and(bool_matrix1, bool_matrix2))
    FP = np.sum(np.logical_and(np.logical_not(bool_matrix1), bool_matrix2))
    FN = np.sum(np.logical_and(bool_matrix1, np.logical_not(bool_matrix2)))
    TN = np.sum(np.logical_and(np.logical_not(bool_matrix1), np.logical_not(bool_matrix2)))

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0

    return precision, recall, f1_score, accuracy

def structural_hamming_distance(matrix1, matrix2):
    assert matrix1.shape == matrix2.shape, "Matrices must have the same shape"

    bool_matrix1 = matrix1 != 0
    bool_matrix2 = matrix2 != 0

    distance = np.sum(bool_matrix1 != bool_matrix2)

    return distance

# 定义处理所有数据集的函数
def process_all_datasets(path_input, path_groundtruth, output_file='results.csv', threshold=0.1):
    files_input_name = [f for f in listdir(path_input) if isfile(join(path_input, f)) and not f.startswith('.')]
    results = []

    for file_input_name in files_input_name:
        timeseries_file = join(path_input, file_input_name)
        idx_ground_truth_file = file_input_name.split('timeseries')[1].split('.csv')[0]
        file_ground_truth_name = "sim" + idx_ground_truth_file + "_gt_processed.csv"
        groundtruth_file = join(path_groundtruth, file_ground_truth_name)

        result = process_dataset(timeseries_file, groundtruth_file, threshold)
        result["dataset"] = file_input_name
        results.append(result)

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)

    # # Calculate variance and standard deviation for each metric
    # metrics = ["SHD", "error rate", "accuracy", "precision", "recall", "f1score"]
    # variance = {metric: np.var(results_df[metric]) for metric in metrics}
    # std_deviation = {metric: np.std(results_df[metric]) for metric in metrics}

    # # Add variance and standard deviation to the results DataFrame
    # for metric in metrics:
    #     results_df[f"{metric}_variance"] = variance[metric]
    #     results_df[f"{metric}_std_deviation"] = std_deviation[metric]

    # # Save updated results with variance and standard deviation
    # results_df.to_csv(output_file, index=False)

    return results_df

# 路径设置
path_input = './data/data_benchmark_1/fMRI_processed_by_Nauta/returns/'
path_groundtruth = './data/data_benchmark_1/fMRI_processed_by_Nauta/ground_truth/'
output_file = 'results.csv'

# 处理所有数据集并保存结果
results_df = process_all_datasets(path_input, path_groundtruth, output_file, threshold=0.2)
print(results_df)


Estimate VAR coefficients time: 0.0020949840545654297 seconds
precomputation Execution time: 0.002003908157348633 seconds
Measure method: pwling_v2
search causal order Execution time: 0.0009992122650146484 seconds
estimate adjacency matrix Execution time: 0.00699925422668457 seconds
Estimate VAR coefficients time: 0.0020034313201904297 seconds
precomputation Execution time: 0.0019996166229248047 seconds
Measure method: pwling_v2
search causal order Execution time: 0.0006539821624755859 seconds
estimate adjacency matrix Execution time: 0.009284734725952148 seconds
Estimate VAR coefficients time: 0.00099945068359375 seconds
precomputation Execution time: 0.006000041961669922 seconds
Measure method: pwling_v2
search causal order Execution time: 0.0025103092193603516 seconds
estimate adjacency matrix Execution time: 0.02456355094909668 seconds
Estimate VAR coefficients time: 0.0025048255920410156 seconds
precomputation Execution time: 0.00551915168762207 seconds
Measure method: pwling_v2
s

In [3]:

print(np.mean(results_df["f1score"]))
print(np.std(results_df["f1score"]))

0.6197515738247849
0.13946696550768203
